### Data Cleaning
 This notebook dedicates to the cleaning and feature engineering of the original dataset

In [1]:
import numpy as np
import pandas as pd
from uszipcode import SearchEngine
import re

/Users/chrisgao/miniconda3/envs/DriveDeepDive/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('../data/raw/used_car_sales.csv')

In [3]:
#Wanted columns for Dashboard:
selected_columns = ['pricesold', 'yearsold', 'zipcode', 'Mileage', 'Make', 'Year', 'BodyType']
df_selected = df[selected_columns]
df_selected.head()

,pricesold,yearsold,zipcode,Mileage,Make,Year,BodyType
0,7500,2020,786**,84430,Ford,1988,Sedan
1,15000,2019,81006,0,Replica/Kit Makes,1958,Convertible
2,8750,2020,33449,55000,Jaguar,1995,Convertible
3,11600,2019,07852,97200,Ford,1968,Coupe
4,44000,2019,07728,40703,Porsche,2002,Coupe


In [4]:
df_selected_us = df[df['zipcode'].astype(str).str.len() == 5]

### Body Type Cleaning

In [5]:
unique_body = df_selected_us['BodyType'].unique()
unique_body

array(['Sedan', 'Convertible', 'Coupe', ..., 'Rugged', 'Mid-Size',
       'Travelall'], dtype=object)

In [6]:
def categorize_bodytype(bodytype):
    if pd.isna(bodytype):
        #make na OTHER
        return 'Other'
    bodytype = bodytype.lower()
    
    #regex patterns by bodytype
    patterns = {
    'Sedan': r'\b(4dr|4 door|sedan|saloon)\b',
    'Coupe': r'\b(2dr|2 door|coupe|hardtop|targa)\b',
    'Truck': r'\b(truck|pickup|flatbed|stepside|extended cab|crew cab|double cab|single cab|quad cab|utility|ute|service truck)\b',
    'Van': r'\b(van|wagon|cargo|passenger|minivan)\b',
    'Convertible': r'\b(convertible|cabriolet|roadster|spyder|targa)\b',
    'SUV': r'\b(suv|crossover|jeep)\b',
    'Hatchback': r'\b(hatchback|liftback|fastback)\b',
    'Station Wagon': r'\b(station wagon|estate)\b',
    'Motorcycle': r'\b(motorcycle|bike|scooter)\b'
}
    
    for category, pattern in patterns.items():
        if re.search(pattern, bodytype):
            return category

    #'Other' if no pattern
    return 'Other'

df_selected_us['BodyType'] = df_selected_us['BodyType'].apply(categorize_bodytype)

/var/folders/gh/09qn1fb15y3g9q_xd8w2mb8m0000gn/T/ipykernel_73340/3640828165.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_us['BodyType'] = df_selected_us['BodyType'].apply(categorize_bodytype)


In [7]:
df_selected_us['BodyType'].value_counts()

BodyType
Other          23980
Sedan          21430
Coupe          19432
Truck          16980
SUV            15363
Convertible    12489
Van             7146
Hatchback       3668
Motorcycle         5
Name: count, dtype: int64

### Filtering: Price Mileage and Year

In [8]:
#Filtering of price, mileage, and year
filtered_df = df_selected_us[
    (df_selected_us['Mileage'] > 20) & 
    (df_selected_us['Year'] >= 1960) & 
    (df_selected_us['pricesold'] > 500)
].dropna()
filtered_df = filtered_df[(filtered_df['Make'] != 'Other') & (filtered_df['BodyType'] != 'Other')]
filtered_df = filtered_df.reset_index(drop=True)

### Mileage Binning

In [9]:
#Binning Sales Value and Mileage values
mileage_bins = [0, 20000, 50000, 100000, float('inf')]
mileage_labels = ['New', 'Slightly Used', 'Moderately Used', 'Very Used']
filtered_df['Quality'] = pd.cut(filtered_df['Mileage'], bins=mileage_bins, labels=mileage_labels, right=False)

#bin 'pricesold' as 'Class' 
price_bins = [0, 30000, 80000, float('inf')]
price_labels = ['Economy', 'Mid Range', 'Luxury']
filtered_df['Class'] = pd.cut(filtered_df['pricesold'], bins=price_bins, labels=price_labels, right=False)

In [10]:
filtered_df

,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType,Quality,Class
0,137178,7500,2020,786**,84430,Ford,Mustang,1988,LX,5.0L Gas V8,Sedan,0,RWD,Moderately Used,Economy
1,119660,8750,2020,33449,55000,Jaguar,XJS,1995,2+2 Cabriolet,4.0L In-Line 6 Cylinder,Convertible,6,RWD,Moderately Used,Economy
2,80773,11600,2019,07852,97200,Ford,Mustang,1968,Stock,289 cu. in. V8,Coupe,8,RWD,Moderately Used,Economy
3,64287,44000,2019,07728,40703,Porsche,911,2002,Turbo X-50,3.6L,Coupe,6,AWD,Slightly Used,Mid Range
4,158271,20000,2020,333**,51674,Jeep,Wrangler,2015,SPORT,3.6L Flexible V6,SUV,6,4WD,Moderately Used,Economy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54673,8426,27900,2019,23936,80540,Ford,F-250,2002,7.3L Diesel 4x4 1owner Lariat,7.3 Turbo Diesel,Truck,8,4WD-3.73 Limited Slip Axle,Moderately Used,Economy
54674,10511,3500,2019,94952,61413,Dodge,Other Pickups,1966,Custom Sport Special,440,Truck,8,RWD,Moderately Used,Economy
54675,87007,6700,2019,95407,76413,Ford,Bronco,1979,Custom,400 V8,SUV,8,4WD,Moderately Used,Economy
54676,29096,2280,2019,92131,164337,BMW,3-Series,2000,328ci,M52TU 2.8L,Coupe,6,RWD,Very Used,Economy


### Mapping Zipcode into States in the US

In [11]:
def modify_zipcode(zipcode):
    pattern = r'\d{3}\*\*'
    if re.match(pattern, zipcode):
        return re.sub(r'\*\*', '01', zipcode)
    else:
        return zipcode

# Applying the modification function to the DataFrame
filtered_df['modified_zipcode'] = filtered_df['zipcode'].apply(modify_zipcode)
filtered_df

,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType,Quality,Class,modified_zipcode
0,137178,7500,2020,786**,84430,Ford,Mustang,1988,LX,5.0L Gas V8,Sedan,0,RWD,Moderately Used,Economy,78601
1,119660,8750,2020,33449,55000,Jaguar,XJS,1995,2+2 Cabriolet,4.0L In-Line 6 Cylinder,Convertible,6,RWD,Moderately Used,Economy,33449
2,80773,11600,2019,07852,97200,Ford,Mustang,1968,Stock,289 cu. in. V8,Coupe,8,RWD,Moderately Used,Economy,07852
3,64287,44000,2019,07728,40703,Porsche,911,2002,Turbo X-50,3.6L,Coupe,6,AWD,Slightly Used,Mid Range,07728
4,158271,20000,2020,333**,51674,Jeep,Wrangler,2015,SPORT,3.6L Flexible V6,SUV,6,4WD,Moderately Used,Economy,33301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54673,8426,27900,2019,23936,80540,Ford,F-250,2002,7.3L Diesel 4x4 1owner Lariat,7.3 Turbo Diesel,Truck,8,4WD-3.73 Limited Slip Axle,Moderately Used,Economy,23936
54674,10511,3500,2019,94952,61413,Dodge,Other Pickups,1966,Custom Sport Special,440,Truck,8,RWD,Moderately Used,Economy,94952
54675,87007,6700,2019,95407,76413,Ford,Bronco,1979,Custom,400 V8,SUV,8,4WD,Moderately Used,Economy,95407
54676,29096,2280,2019,92131,164337,BMW,3-Series,2000,328ci,M52TU 2.8L,Coupe,6,RWD,Very Used,Economy,92131


In [12]:
search = SearchEngine()
def map_zipcode_to_state(zipcode):
    result = search.by_zipcode(zipcode)
    if result and result.state:
        return result.state
    else:
        return "Invalid or Unknown Zipcode"

filtered_df['state'] = filtered_df['modified_zipcode'].apply(map_zipcode_to_state)


In [13]:
df_processed = filtered_df[filtered_df['state'] != 'Invalid or Unknown Zipcode']
df_processed

,ID,pricesold,yearsold,zipcode,Mileage,Make,Model,Year,Trim,Engine,BodyType,NumCylinders,DriveType,Quality,Class,modified_zipcode,state
1,119660,8750,2020,33449,55000,Jaguar,XJS,1995,2+2 Cabriolet,4.0L In-Line 6 Cylinder,Convertible,6,RWD,Moderately Used,Economy,33449,FL
2,80773,11600,2019,07852,97200,Ford,Mustang,1968,Stock,289 cu. in. V8,Coupe,8,RWD,Moderately Used,Economy,07852,NJ
3,64287,44000,2019,07728,40703,Porsche,911,2002,Turbo X-50,3.6L,Coupe,6,AWD,Slightly Used,Mid Range,07728,NJ
4,158271,20000,2020,333**,51674,Jeep,Wrangler,2015,SPORT,3.6L Flexible V6,SUV,6,4WD,Moderately Used,Economy,33301,FL
5,72418,14100,2019,07014,109500,Jeep,Wrangler,2012,Unlimited,3.6L,SUV,6,4WD,Very Used,Economy,07014,NJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54673,8426,27900,2019,23936,80540,Ford,F-250,2002,7.3L Diesel 4x4 1owner Lariat,7.3 Turbo Diesel,Truck,8,4WD-3.73 Limited Slip Axle,Moderately Used,Economy,23936,VA
54674,10511,3500,2019,94952,61413,Dodge,Other Pickups,1966,Custom Sport Special,440,Truck,8,RWD,Moderately Used,Economy,94952,CA
54675,87007,6700,2019,95407,76413,Ford,Bronco,1979,Custom,400 V8,SUV,8,4WD,Moderately Used,Economy,95407,CA
54676,29096,2280,2019,92131,164337,BMW,3-Series,2000,328ci,M52TU 2.8L,Coupe,6,RWD,Very Used,Economy,92131,CA


In [15]:
df_processed.to_csv('../data/preprocessed/processed_data.csv')